In [14]:
import pandas as pd
import tushare as ts
import plotly.express as px
import plotly.graph_objects as go
import datetime
pro = ts.pro_api()
today = datetime.datetime.today()
cur_year = str(today.year)

In [15]:
comp_dicts = {'603589':'口子窖',
            '000596':'古井贡酒',
            '603189':'迎驾贡酒',
            '600199':'金种子酒'}

target_code = '603589.sh'
# target_code = '002304.sz' # y洋河

In [16]:
for k,v in list(comp_dicts.items()):
    if "." in k: continue
    surfix = '.SH' if k.startswith('6') else '.SZ'
    comp_dicts.pop(k)
    comp_dicts[k+surfix] = v
    
ts_codes = ','.join(comp_dicts.keys())

In [17]:
def data_process(df):
    df = df.copy()
#     df = df[df.end_date.apply(lambda x:(x.endswith('1231') or x > cur_year) and x>'2010')] # 保留当前年份报告期数据
    df = df[df.end_date.apply(lambda x:(x.endswith('1231')) and x>'2010')]
    df['end_date'] = df['end_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:])

    df['sumv'] = df.fillna(0).sum(axis=1)
    df = df.groupby(['ts_code','end_date']).apply(lambda t:t[t.sumv==t.sumv.max()]).reset_index(drop=True).groupby(['ts_code','end_date']).head(1)
    df.index = df['end_date']
    df = df.drop('end_date', axis=1)
    return df

In [57]:
def get_data(ts_code):
    balancesheet = pro.balancesheet(ts_code=target_code)
    balancesheet = data_process(balancesheet)

    income = pro.income(ts_code=target_code)
    income = data_process(income)

    cashflow = pro.cashflow(ts_code=target_code)
    cashflow = data_process(cashflow)
    
    fina_indicator = pro.fina_indicator(ts_code=target_code)
    fina_indicator = data_process(fina_indicator)

    df1 = pd.merge(balancesheet, income, on='end_date', suffixes=('','_inc'))
    df1 = pd.merge(df1, cashflow, on='end_date', suffixes=('','_cas'))
    df1 = pd.merge(df1, fina_indicator, on='end_date', suffixes=('','_fina'))
    
    df1 = df1.fillna(0)
    return df1

In [58]:
data_df = get_data(target_code)

# ROE 分析

## ROE

In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.roe,name='roe'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.roe_dt,name='roe_dt'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.roa,name='roa'))
# fig.add_trace(go.Scatter(x=data_df.index, y=data_df.roe_dt,name='roe_dt'))
fig.show()

## 净利率 & 毛利率

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.netprofit_margin,name='netprofit_margin'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.grossprofit_margin,name='grossprofit_margin', yaxis='y2'))
fig.update_layout(yaxis2=dict(anchor='x', overlaying='y', side='right')#设置坐标轴的格式，一般次坐标轴在右侧
)
fig.show()

## 周转率

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.assets_turn,name='assets_turn'))
# fig.add_trace(go.Scatter(x=data_df.index, y=data_df.grossprofit_margin,name='grossprofit_margin', yaxis='y2'))
# fig.update_layout(yaxis2=dict(anchor='x', overlaying='y', side='right')#设置坐标轴的格式，一般次坐标轴在右侧)
fig.show()


## 资产负债率

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.debt_to_assets,name='debt_to_assets'))
# fig.add_trace(go.Scatter(x=data_df.index, y=data_df.grossprofit_margin,name='grossprofit_margin', yaxis='y2'))
# fig.update_layout(yaxis2=dict(anchor='x', overlaying='y', side='right')#设置坐标轴的格式，一般次坐标轴在右侧)
fig.show()



### 有息负债率

In [60]:
cols_yx_debts = {
    "lt_borr":"长期借款",
"st_borr":"短期借款",
"cb_borr":"向中央银行借款",
"trading_fl":"交易性金融负债",
"sold_for_repur_fa":"卖出回购金融资产款",
"st_bonds_payable":"应付短期债券",
"non_cur_liab_due_1y":"一年内到期的非流动负债",
"oth_cur_liab":"其他流动负债",
"bond_payable":"应付债券",
"oth_ncl":"其他非流动负债",
}
data_df['debts_yx'] = data_df[cols_yx_debts.keys()].sum(axis=1)

In [61]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df['debts_yx']/ data_df['total_assets'],name='有息负债率'))
fig.show()



### 去掉商誉的有息负债率

In [62]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df['debts_yx'] / (data_df['total_assets'] - data_df['goodwill']),name='除商誉的有息负债率'))
fig.show()



# 安全性

## 现金及等价物/有息负债

In [63]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df['money_cap']/data_df['debts_yx'],name='有息负债率'))
fig.show()


# 成长性

## 营收、扣非、净利润增长

In [85]:
data_df['profit_dedt_yoy'] = data_df.profit_dedt/data_df.profit_dedt.shift(1)
data_df['revenue_yoy'] = data_df.revenue/data_df.revenue.shift(1)


In [91]:
fig = go.Figure()

fig.add_trace(go.Bar(x=data_df.index, y=data_df.total_revenue, name='total_revenue',offsetgroup=0))
fig.add_trace(go.Bar(x=data_df.index, y=data_df.n_income,name='n_income', offsetgroup=1))

fig.add_trace(go.Scatter(x=data_df.index, y=data_df.revenue_yoy,name='revenue_yoy', yaxis='y2'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.profit_dedt_yoy,name='扣非净利润增长', yaxis='y2'))

fig.update_layout(yaxis2=dict(anchor='x', overlaying='y', side='right'))#设置坐标轴的格式，一般次坐标轴在右侧
fig.show()

## 总资产、净资产增长率

In [77]:
data_df['total_ass_yoy'] = data_df.total_assets/data_df.total_assets.shift(1)
data_df['total_hldr_eqy_inc_min_int_yoy'] = data_df.total_hldr_eqy_inc_min_int/data_df.total_hldr_eqy_inc_min_int.shift(1)


In [80]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_assets,name='total_assets'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_ass_yoy , yaxis='y2',name='total_ass_yoy'))

fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_hldr_eqy_inc_min_int,name='所有者权益'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_hldr_eqy_inc_min_int_yoy , yaxis='y2',name='所有者权益yoy'))

fig.update_layout(yaxis2=dict(anchor='x', overlaying='y', side='right'))#设置坐标轴的格式，一般次坐标轴在右侧
fig.show()

# 资产负债表

## 资产负债表中的几个比率

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_assets))
fig.show()

## 货币资金的比率

In [25]:
data_df['money_cap_ratio'] = data_df['money_cap']/data_df['total_assets']
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.money_cap_ratio))
fig.show()

## 经营资产比率

In [26]:
cols_op = {"notes_receiv":"应收票据",
"accounts_receiv":"应收账款",
"oth_receiv":"其他应收款",
"prepayment":"预付款项",
"div_receiv":"应收股利",
"int_receiv":"应收利息",
"inventories":"存货",
"produc_bio_assets":"生产性生物资产"}

In [35]:
data_df['op_ratio'] = data_df[cols_op.keys()].sum(axis=1)/data_df['total_assets']
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.op_ratio))
fig.show()

In [37]:
fig = go.Figure()
for col in cols_op:
    fig.add_trace(go.Bar(x=data_df.index, y=data_df[col], name=cols_op[col]))

# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

## 生产资产

In [ ]:
cols_prod = {"fix_assets":"固定资产",
"cip":"在建工程",
"const_materials":"工程物资",
"intan_assets":"无形资产",
"goodwill":"商誉",
"lt_amor_exp":"长期待摊费用",
"defer_tax_assets":"递延所得税资产"}
# ？递延所得税资产 or 负债？

In [ ]:
data_df['prod_ratio'] = data_df[cols_prod.keys()].sum(axis=1)/data_df['total_assets']
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.end_date, y=data_df.prod_ratio))
fig.show()

## 非主营相关资产

In [ ]:
cols_oth_ass = {
"trad_asset":"交易性金融资产",
"pur_resale_fa":"买入返售金融资产",
"oth_cur_assets":"其他流动资产",
"fa_avail_for_sale":"可供出售金融资产",
"htm_invest":"持有至到期投资",
"lt_eqt_invest":"长期股权投资",
"invest_real_estate":"投资性房地产"}

In [ ]:
data_df['oth_ass_ratio'] = data_df[cols_oth_ass.keys()].sum(axis=1)/data_df['total_assets']
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.end_date, y=data_df.oth_ass_ratio))
fig.show()

In [ ]:
data_df = data_df.fillna(0)

In [ ]:
fig = go.Figure()
for col in cols_oth_ass:
    fig.add_trace(go.Bar(x=data_df.end_date, y=data_df[col], name=cols_oth_ass[col]))

# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
83/2265

In [ ]:
## 2019理财收益
35/2335
## ？？大股东占用资金？

In [ ]:
# 除商誉外的资产负债率
# 安全性

# 利润表

In [65]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_revenue, name='total_revenue'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.operate_profit, name='operate_profit'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.n_income,name='n_income'))
fig.show()
# px.line(data_df,x='end_date',y=['total_revenue','operate_profit','n_income'])

# 现金流量表

In [32]:
## 使用bar，对比分组数据
fig = go.Figure()
fig.add_trace(go.Bar(x=data_df.index, y=data_df.n_cashflow_act, offsetgroup=0, name='经营活动现金流净额'))
fig.add_trace(go.Bar(x=data_df.index, y=-data_df.n_cashflow_inv_act, offsetgroup=1, name='投资活动现金流净额'))
fig.add_trace(go.Bar(x=data_df.index, y=-data_df.n_cash_flows_fnc_act, offsetgroup=1,name='融资活动现金流净额'))
fig.show()
# px.line(data_df,x='end_date',y=['total_revenue','operate_profit','n_income'])

In [33]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.c_fr_sale_sg, name='销售收现'))
fig.show()


In [34]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_revenue, name='销售收现'))
fig.show()


In [43]:
text = """
lt_borr	长期借款
st_borr	短期借款
cb_borr	向中央银行借款
trading_fl	交易性金融负债
sold_for_repur_fa	卖出回购金融资产款
st_bonds_payable	应付短期债券
non_cur_liab_due_1y	一年内到期的非流动负债
oth_cur_liab	其他流动负债
bond_payable	应付债券
oth_ncl	其他非流动负债
"""
for line in text.split("\n"):
    if "\t" not in line:continue
    k,c = line.split("\t")
    print(f'"{k}":"{c}",')

"lt_borr":"长期借款",
"st_borr":"短期借款",
"cb_borr":"向中央银行借款",
"trading_fl":"交易性金融负债",
"sold_for_repur_fa":"卖出回购金融资产款",
"st_bonds_payable":"应付短期债券",
"non_cur_liab_due_1y":"一年内到期的非流动负债",
"oth_cur_liab":"其他流动负债",
"bond_payable":"应付债券",
"oth_ncl":"其他非流动负债",
